# Классификация текстов Multilabel

## Загрузка данных

In [ ]:
train = pd.read_csv('train_data.csv')
test = pd.read_csv('test_data.csv')
classes = pd.read_csv('classes.csv')
sample_submission = pd.read_csv('sample_submission.csv')

actual_data_1 = pd.read_csv('actual_data_1.csv')

## Импорт библиотек

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import recall_score, precision_score
import io

In [ ]:
!pip install catboost

In [ ]:
!pip install pymorphy2

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt

from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import f1_score
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score

In [ ]:
!pip install scikit-optimize
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer

## Векторизация слов

In [ ]:
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import TweetTokenizer

In [ ]:
stemmer = SnowballStemmer('russian')
tokenizer = TweetTokenizer()

tfidf = TfidfVectorizer(max_df=0.3,
                        min_df=3,
                       tokenizer=lambda text: stemm_tokenizer(text, stemmer, tokenizer).split())


tfidf.fit(data.text.dropna())

### Нужный блок

In [ ]:
actual_data_preproc['Target'] = actual_data_preproc['0'] + actual_data_preproc['1'] + actual_data_preproc['2'] + actual_data_preproc['3'] + actual_data_preproc['4']

In [ ]:
vectorizer = TfidfVectorizer()

vectorizer.fit(actual_data_preproc['Документ'].values)

In [ ]:
to_vec = lambda x,  max_: [1 if i in x else 0 for i in range(max_)]

In [ ]:
classes_obj = np.array([to_vec(actual_data_preproc.dropna()['0'].values[i], 195) for i in range(len(actual_data_preproc))])
classes_func = np.array([to_vec(actual_data_preproc.dropna()['1'].values[i], 193) for i in range(len(actual_data_preproc))])
classes_proc = np.array([to_vec(actual_data_preproc.dropna()['2'].values[i], 193) for i in range(len(actual_data_preproc))])
classes_lim = np.array([to_vec(actual_data_preproc.dropna()['3'].values[i], 193) for i in range(len(actual_data_preproc))])
classes_struct = np.array([to_vec(actual_data_preproc.dropna()['4'].values[i], 193) for i in range(len(actual_data_preproc))])

In [ ]:
from preprocessing import preprocess

def form_vector(vocab,x):
    
    ans = np.zeros(len(vocab))
    m = -1
    for i,h in enumerate(vocab):
        if h == x:
            m = i
            break
    if m>=0:
        ans[m] = 1
    return ans

def one_hot(vocab, X):
    '''
    Returns One-Hot-encoded sequence for class-label

    Parameters:
    -----------
    vocab: dict
        Dictionary with structure {num_key: list_of_objects} 
    x: numpy.array()
        Numpy.array() with shape (-1, 1) with lemmatized values

    Returns:
    -------
    vecs: List
        List of one-hot vectors
    '''
    vecs = []
    for x in X:
        vecs.append(form_vector(vocab,x).reshape(-1,1))
    return vecs

f = lambda x: " ".join(x) + " "

vocab = pd.Series(list(set(actual_data_preproc.Раздел)))
vocab = preprocess(vocab)
for i in vocab:
  print(i)

vocab = [[f(x)] for x in vocab]
n = actual_data_preproc.Раздел.apply(lemmatize)

razdel = one_hot(vocab, np.array(n).reshape(-1,1))
OH = np.concatenate(razdel,axis = 1)

## Обучение модели

### Обучение obj

In [ ]:
X = actual_data_preproc['Документ'].values
Y = classes_obj

In [ ]:
pipe = OneVsRestClassifier(LGBMClassifier())

model_obj = BayesSearchCV(
  pipe,
  {
      'estimator__Bosting_type': Categorical(['gbdt', 'dart', 'goss']),
      'estimator__num_leaves': Integer(21, 51),
      'estimator__max_depth': Integer(2, 10),
      'estimator__learning_rate': Real(1e-03, 1e-01, prior = 'log-uniform'),
      'estimator__class_weight': Categorical([None, 'balanced']),
      'estimator__reg_alpha': Real(1e-05, 2, prior = 'log-uniform'),
      'estimator__reg_lambda': Real(1e-05, 2, prior = 'log-uniform'),
      'estimator__min_split_gain ': Real(0, 1),
      'estimator__min_child_weight': Real(1e-05, 1e-01),
      'estimator__min_child_samples ': Integer(5, 30)
  },
  optimizer_kwargs = None,
  n_iter=25,
  scoring = 'f1_micro',
  cv = 5
)

In [ ]:
model_obj.fit(vectorizer.transform(X), Y)

In [ ]:
print("val. score: %s" % model_obj.best_score_)
print("best parameters: %s" % str(model_obj.best_params_))

val. score: 0.7400443993908475
best parameters: OrderedDict([('estimator__Bosting_type', 'dart'), ('estimator__class_weight', None), ('estimator__learning_rate', 0.040296262772471385), ('estimator__max_depth', 10), ('estimator__min_child_samples ', 21), ('estimator__min_child_weight', 1e-05), ('estimator__min_split_gain ', 1.0), ('estimator__num_leaves', 21), ('estimator__reg_alpha', 2.0), ('estimator__reg_lambda', 2.0)])


In [ ]:
clf = OneVsRestClassifier(LGBMClassifier())
cv_res = cross_val_score(clf, vectorizer.transform(X), Y, cv = 5, scoring = 'f1_micro')

In [ ]:
print(cv_res)
print(cv_res.mean())

[0.62285714 0.60655738 0.85158151 0.79283887 0.65365854]
0.7054986879375622


### Обучение func

In [ ]:
X = actual_data_preproc['Документ'].values
Y = classes_func

In [ ]:
pipe = OneVsRestClassifier(LGBMClassifier())

model_func = BayesSearchCV(
  pipe,
  {
      'estimator__Bosting_type': Categorical(['gbdt', 'dart', 'goss']),
      'estimator__num_leaves': Integer(21, 51),
      'estimator__max_depth': Integer(2, 10),
      'estimator__learning_rate': Real(1e-03, 1e-01, prior = 'log-uniform'),
      'estimator__class_weight': Categorical([None, 'balanced']),
      'estimator__reg_alpha': Real(1e-05, 2, prior = 'log-uniform'),
      'estimator__reg_lambda': Real(1e-05, 2, prior = 'log-uniform'),
      'estimator__min_split_gain ': Real(0, 1),
      'estimator__min_child_weight': Real(1e-05, 1e-01),
      'estimator__min_child_samples ': Integer(5, 30)
  },
  optimizer_kwargs = None,
  n_iter=25,
  scoring = 'f1_micro',
  cv = 5
)

In [ ]:
model_func.fit(vectorizer.transform(X), Y)

In [ ]:
print("val. score: %s" % model_func.best_score_)
print("best parameters: %s" % str(model_func.best_params_))

val. score: 0.7706621145960036
best parameters: OrderedDict([('estimator__Bosting_type', 'dart'), ('estimator__class_weight', None), ('estimator__learning_rate', 0.00286091922928078), ('estimator__max_depth', 3), ('estimator__min_child_samples ', 25), ('estimator__min_child_weight', 0.016309427559150363), ('estimator__min_split_gain ', 0.7537680582191356), ('estimator__num_leaves', 47), ('estimator__reg_alpha', 0.0021726168605991658), ('estimator__reg_lambda', 0.2823979958319112)])


In [ ]:
clf = OneVsRestClassifier(LGBMClassifier())
cv_res = cross_val_score(clf, vectorizer.transform(X), Y, cv = 5, scoring = 'f1_micro')

In [ ]:
print(cv_res)
print(cv_res.mean())

[0.69711538 0.79012346 0.84444444 0.77142857 0.74891775]
0.7704059212392546


### Обучение proc

In [ ]:
X = actual_data_preproc['Документ'].values
Y = classes_proc

In [ ]:
pipe = OneVsRestClassifier(LGBMClassifier())

model_proc = BayesSearchCV(
  pipe,
  {
      'estimator__Bosting_type': Categorical(['gbdt', 'dart', 'goss']),
      'estimator__num_leaves': Integer(21, 51),
      'estimator__max_depth': Integer(2, 10),
      'estimator__learning_rate': Real(1e-03, 1e-01, prior = 'log-uniform'),
      'estimator__class_weight': Categorical([None, 'balanced']),
      'estimator__reg_alpha': Real(1e-05, 2, prior = 'log-uniform'),
      'estimator__reg_lambda': Real(1e-05, 2, prior = 'log-uniform'),
      'estimator__min_split_gain ': Real(0, 1),
      'estimator__min_child_weight': Real(1e-05, 1e-01),
      'estimator__min_child_samples ': Integer(5, 30)
  },
  optimizer_kwargs = None,
  n_iter=25,
  scoring = 'f1_micro',
  cv = 5
)

In [ ]:
model_proc.fit(vectorizer.transform(X), Y)

In [ ]:
print("val. score: %s" % model_proc.best_score_)
print("best parameters: %s" % str(model_proc.best_params_))

val. score: 0.6878013953682423
best parameters: OrderedDict([('estimator__Bosting_type', 'dart'), ('estimator__class_weight', None), ('estimator__learning_rate', 0.1), ('estimator__max_depth', 7), ('estimator__min_child_samples ', 26), ('estimator__min_child_weight', 0.1), ('estimator__min_split_gain ', 1.0), ('estimator__num_leaves', 51), ('estimator__reg_alpha', 1e-05), ('estimator__reg_lambda', 1e-05)])


In [ ]:
clf = OneVsRestClassifier(LGBMClassifier())
cv_res = cross_val_score(clf, vectorizer.transform(X), Y, cv = 5, scoring = 'f1_micro')

In [ ]:
print(cv_res)
print(cv_res.mean())

[0.63937622 0.6427289  0.8        0.72627737 0.62752294]
0.6871810862427148


### Обучение lim

In [ ]:
X = actual_data_preproc['Документ'].values
Y = classes_lim

In [ ]:
pipe = OneVsRestClassifier(LGBMClassifier())

model_lim = BayesSearchCV(
  pipe,
  {
      'estimator__Bosting_type': Categorical(['gbdt', 'dart', 'goss']),
      'estimator__num_leaves': Integer(21, 51),
      'estimator__max_depth': Integer(2, 10),
      'estimator__learning_rate': Real(1e-03, 1e-01, prior = 'log-uniform'),
      'estimator__class_weight': Categorical([None, 'balanced']),
      'estimator__reg_alpha': Real(1e-05, 2, prior = 'log-uniform'),
      'estimator__reg_lambda': Real(1e-05, 2, prior = 'log-uniform'),
      'estimator__min_split_gain ': Real(0, 1),
      'estimator__min_child_weight': Real(1e-05, 1e-01),
      'estimator__min_child_samples ': Integer(5, 30)
  },
  optimizer_kwargs = None,
  n_iter=25,
  scoring = 'f1_micro',
  cv = 5
)

In [ ]:
model_lim.fit(vectorizer.transform(X), Y)

In [ ]:
print("val. score: %s" % model_lim.best_score_)
print("best parameters: %s" % str(model_lim.best_params_))

val. score: 0.9208217056716477
best parameters: OrderedDict([('estimator__Bosting_type', 'dart'), ('estimator__class_weight', None), ('estimator__learning_rate', 0.1), ('estimator__max_depth', 4), ('estimator__min_child_samples ', 6), ('estimator__min_child_weight', 1e-05), ('estimator__min_split_gain ', 0.03861981093521761), ('estimator__num_leaves', 21), ('estimator__reg_alpha', 1e-05), ('estimator__reg_lambda', 1e-05)])


In [ ]:
clf = OneVsRestClassifier(LGBMClassifier())
cv_res = cross_val_score(clf, vectorizer.transform(X), Y, cv = 5, scoring = 'f1_micro')

In [ ]:
print(cv_res)
print(cv_res.mean())

[0.90697674 0.85651214 0.99334812 0.98454746 0.84395604]
0.9170681012180865


### Обучение struct

In [ ]:
X = actual_data_preproc['Документ'].values
Y = classes_struct

In [ ]:
pipe = OneVsRestClassifier(LGBMClassifier())

model_struct = BayesSearchCV(
  pipe,
  {
      'estimator__Bosting_type': Categorical(['gbdt', 'dart', 'goss']),
      'estimator__num_leaves': Integer(21, 51),
      'estimator__max_depth': Integer(2, 10),
      'estimator__learning_rate': Real(1e-03, 1e-01, prior = 'log-uniform'),
      'estimator__class_weight': Categorical([None, 'balanced']),
      'estimator__reg_alpha': Real(1e-05, 2, prior = 'log-uniform'),
      'estimator__reg_lambda': Real(1e-05, 2, prior = 'log-uniform'),
      'estimator__min_split_gain ': Real(0, 1),
      'estimator__min_child_weight': Real(1e-05, 1e-01),
      'estimator__min_child_samples ': Integer(5, 30)
  },
  optimizer_kwargs = None,
  n_iter=25,
  scoring = 'f1_micro',
  cv = 5
)

In [ ]:
model_struct.fit(vectorizer.transform(X), Y)

In [ ]:
print("val. score: %s" % model_struct.best_score_)
print("best parameters: %s" % str(model_struct.best_params_))

val. score: 0.8919920623677218
best parameters: OrderedDict([('estimator__Bosting_type', 'gbdt'), ('estimator__class_weight', None), ('estimator__learning_rate', 0.07347761922915837), ('estimator__max_depth', 6), ('estimator__min_child_samples ', 29), ('estimator__min_child_weight', 0.009009810941141035), ('estimator__min_split_gain ', 0.3246035834206646), ('estimator__num_leaves', 46), ('estimator__reg_alpha', 0.008784182627432172), ('estimator__reg_lambda', 0.03449797384143123)])


In [ ]:
clf = OneVsRestClassifier(LGBMClassifier())
cv_res = cross_val_score(clf, vectorizer.transform(X), Y, cv = 5, scoring = 'f1_micro')

In [ ]:
print(cv_res)
print(cv_res.mean())

[0.85943775 0.8782936  0.94570136 0.92290249 0.83939039]
0.8891451181349561


## Предикт

In [ ]:
from preprocessing import preprocess

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
texts,word_freq = preprocess(test['Документ'])

In [ ]:
test_preproc = test.copy()

In [ ]:
test_preproc['Документ'] = texts

In [ ]:
test_preproc.loc[1174, 'Doc'] = ['']
test_preproc.loc[1535, 'Doc'] = ['']

In [ ]:
for i in range(len(test_preproc)):
  test_preproc.loc[i, 'Doc'] = ' '.join(test_preproc.loc[i, 'Doc'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
submission = {}
submission = pd.DataFrame(submission)

In [ ]:
X = vectorizer.transform(test_preproc['Doc'].values)

In [ ]:
submission['id'] = test_preproc['id']
submission['file_id'] = test_preproc['file_id']

In [ ]:
preds = model_obj.predict(X)
submission['0'] = one_hot_decoder(preds, classes['0'])

In [ ]:
preds = model_func.predict(X)
submission['1'] = one_hot_decoder(preds, classes['1'])

In [ ]:
preds = model_proc.predict(X)
submission['2'] = one_hot_decoder(preds, classes['2'])

In [ ]:
preds = model_lim.predict(X)
submission['3'] = one_hot_decoder(preds, classes['3'])

In [ ]:
preds = model_struct.predict(X)
submission['4'] = one_hot_decoder(preds, classes['4'])

In [ ]:
submission = submission.replace('','Не опознано')

In [ ]:
submission

In [ ]:
submission.to_csv('submission.csv', encoding='utf-8-sig', sep=',', index=False)